In [1]:
from bs4 import BeautifulSoup
import pymongo
import requests
import re
import sys

In [2]:
# Create mongodb connection
from pymongo import MongoClient
connection = MongoClient('localhost', 27017)
database = connection.housing_data

table = database.etuovi

#table.drop() to drop the collection 'housing_data'

cookie = {'org.springframework.mobile.device.site.CookieSitePreferenceRepository.SITE_PREFERENCE':'NORMAL','JSESSIONID':'183EE2F12C128B603790A0D5D1558022','evid_0006':'87ccc59d-4b80-489f-98cf-be92d6591fa6','evid_0006_ref':'false','adptseg_0006':'kv1001#a-kv1002#c-kv1003#d-kv1004#d-kv1005#c-kv1006#a:b-kv1008#c-kv1009#17-kv1010#b-kv1011#f-kv1012#c-kv1013#a-kv1016#1:11:13:2:7-kv1017#2-','adptset_0006':'1','evid_0006_set':'2','__utma':'82691985.304271103.1385626909.1385642039.1385658686.4','__utmb':'82691985.9.8.1385658723878','__utmc':'82691985','__utmz':'82691985.1385626909.1.1.utmcsr=kuluttaja.etuovi.com|utmccn=(referral)|utmcmd=referral|utmcct=/crometapp/product/realties/common/public/frontpage2/search_in_progress_to_vuokraovi.jsp','slbp_80':'2265258762.20480.0000','slbp_443':'2265258762.47873.0000'}

data_added = 0

In [3]:
# Adding all the links to each house listing from the website 
start_page = 1
end_page = 520
Links = []
for page in range(start_page, end_page):
    print ('Parsing page:', str(page))
    html = requests.post('http://www.vuokraovi.com/vuokra-asunnot?page=' + str(page), cookies=cookie, data={'listorder':11})
    
    parsed_html = BeautifulSoup(html.text,'html.parser')                                                                                        
    #print(parsed_html.prettify())
    url_tag = parsed_html.find_all('a', {'class':'list-item-link'})
    for link in url_tag:
        address = link.get('href')
        if '/vuokra-asunto' in address:
            #print(address)
            Links.append(address)
#print (Links)

Parsing page: 1
Parsing page: 2
Parsing page: 3
Parsing page: 4
Parsing page: 5
Parsing page: 6
Parsing page: 7
Parsing page: 8
Parsing page: 9
Parsing page: 10
Parsing page: 11
Parsing page: 12
Parsing page: 13
Parsing page: 14
Parsing page: 15
Parsing page: 16
Parsing page: 17
Parsing page: 18
Parsing page: 19
Parsing page: 20
Parsing page: 21
Parsing page: 22
Parsing page: 23
Parsing page: 24
Parsing page: 25
Parsing page: 26
Parsing page: 27
Parsing page: 28
Parsing page: 29
Parsing page: 30
Parsing page: 31
Parsing page: 32
Parsing page: 33
Parsing page: 34
Parsing page: 35
Parsing page: 36
Parsing page: 37
Parsing page: 38
Parsing page: 39
Parsing page: 40
Parsing page: 41
Parsing page: 42
Parsing page: 43
Parsing page: 44
Parsing page: 45
Parsing page: 46
Parsing page: 47
Parsing page: 48
Parsing page: 49
Parsing page: 50
Parsing page: 51
Parsing page: 52
Parsing page: 53
Parsing page: 54
Parsing page: 55
Parsing page: 56
Parsing page: 57
Parsing page: 58
Parsing page: 59
Parsin

Parsing page: 463
Parsing page: 464
Parsing page: 465
Parsing page: 466
Parsing page: 467
Parsing page: 468
Parsing page: 469
Parsing page: 470
Parsing page: 471
Parsing page: 472
Parsing page: 473
Parsing page: 474
Parsing page: 475
Parsing page: 476
Parsing page: 477
Parsing page: 478
Parsing page: 479
Parsing page: 480
Parsing page: 481
Parsing page: 482
Parsing page: 483
Parsing page: 484
Parsing page: 485
Parsing page: 486
Parsing page: 487
Parsing page: 488
Parsing page: 489
Parsing page: 490
Parsing page: 491
Parsing page: 492
Parsing page: 493
Parsing page: 494
Parsing page: 495
Parsing page: 496
Parsing page: 497
Parsing page: 498
Parsing page: 499
Parsing page: 500
Parsing page: 501
Parsing page: 502
Parsing page: 503
Parsing page: 504
Parsing page: 505
Parsing page: 506
Parsing page: 507
Parsing page: 508
Parsing page: 509
Parsing page: 510
Parsing page: 511
Parsing page: 512
Parsing page: 513
Parsing page: 514
Parsing page: 515
Parsing page: 516
Parsing page: 517
Parsing pa

In [6]:
allApartments = []
apartment = dict()
for page in Links:
    #print(page)
    html = requests.post('http://www.vuokraovi.com' + page, cookies=cookie, data={'listorder':11})
    parsed_html = BeautifulSoup(html.text,'html.parser') 
    #print(parsed_html)
    test_tagth = parsed_html.find_all('th')
    test_tagtd = parsed_html.find_all('td')
    print(len(test_tagtd))
    if(len(test_tagtd) > 0):
        apartment['postal_code'] = str(test_tagtd[0].find('span', {'itemprop':'addressLocality'}).get_text(strip = True)[:5])
        apartment['renter'] = parsed_html.find('b').get_text(strip=True)
    
        for i in range(0,len(test_tagth)):
            if 'Vakuus:' in test_tagth[i].get_text():
                apartment['deposit'] = test_tagtd[i].get_text(strip = True)
            
            if 'Vuokra:' in test_tagth[i].get_text():
                apartment['rent'] = test_tagtd[i].get_text(strip = True) 
            
            if 'Tyyppi:' in test_tagth[i].get_text():
                apartment['type'] = test_tagtd[i].get_text(strip = True)
            
            if 'Huoneiden lukumäärä' in test_tagth[i].get_text():
                apartment['number_of_rooms'] = test_tagtd[i].get_text(strip = True)
            
            if 'Rakennusvuosi:' in test_tagth[i].get_text():
                apartment['built_year'] = test_tagtd[i].get_text(strip = True)
            
            if 'Yleiskunto:' in test_tagth[i].get_text():
                apartment['condition'] = test_tagtd[i].get_text(strip = True)
            
            if 'sallittu:' in test_tagth[i].get_text():
                apartment['allow_pets'] = test_tagtd[i].get_text(strip = True)
            
            if 'pinta-ala:' in test_tagth[i].get_text():
                apartment['total_size'] = test_tagtd[i].get_text(strip = True)
    
    else:
        print ("length")
    #allApartments.append(apartment)
    
    try:
        # Push the collected data to db.
        table.insert_one({
                'postal_code': apartment['postal_code'],
                'renter': apartment['renter'],
                'built_year': apartment['built_year'], 
                'type': apartment['type'],
                'total_size': apartment['total_size'],
                'number_of_rooms': apartment['number_of_rooms'],
                'condition': apartment['condition'],
                'rent': apartment['rent'],
                'deposit': apartment['deposit'],
                'allow_pets': apartment['allow_pets']          
            }) 
        data_added += 1
    except pymongo.errors.PyMongoError:
        pass


47
25
24
23
26
27
25
25
25
44
35
22
28
21
28
27
29
28
51
44
29
47
50
20
20
21
27
27
21
23
23
23
24
21
21
21
22
22
18
22
21
22
21
22
21
22
22
22
22
21
21
21
27
21
21
20
20
20
0
length
43
52
53
51
53
50
50
50
50
50
50
44
44
45
43
31
37
37
24
31
49
22
42
51
18
19
38
39
20
25
25
25
31
23
23
22
25
24
24
24
24
31
38
47
41
40
51
45
41
34
33
42
41
27
55
47
47
47
47
34
47
48
29
33
47
29
29
37
22
22
21
22
37
31
39
38
29
38
25
25
45
29
26
49
21
26
51
0
length
0
length
24
24
24
0
length
28
47
28
44
48
58
27
41
32
24
22
21
22
21
22
21
22
21
22
21
50
52
24
34
47
27
32
23
24
22
32
30
42
23
49
31
52
29
25
21
24
26
24
24
24
25
25
24
24
24
24
25
24
24
24
30
30
31
31
29
35
24
31
37
26
57
57
33
24
23
28
33
24
22
52
22
23
24
27
27
27
37
24
24
22
24
24
40
47
30
31
31
26
28
25
30
30
37
43
35
26
26
31
37
39
29
23
27
27
24
27
24
45
20
20
23
22
22
23
24
30
29
23
27
30
28
32
36
35
33
42
20
20
20
20
46
24
22
22
24
22
25
44
27
27
27
24
26
27
27
27
23
25
24
20
24
24
25
25
24
25
25
25
24
24
24
25
34
25
27
32
25
34
2

21
21
22
21
22
21
21
22
21
22
32
21
20
22
22
29
45
24
23
26
24
28
32
45
23
57
57
24
24
26
29
40
41
19
28
42
28
28
24
24
49
57
57
19
32
57
57
29
33
37
24
24
30
22
35
36
46
43
23
45
24
34
31
20
21
26
26
38
45
25
29
25
24
25
27
24
22
23
24
44
47
27
23
32
24
49
25
27
27
24
24
24
24
24
23
23
23
23
23
34
54
25
29
30
29
28
34
38
24
49
42
44
38
41
23
25
25
21
23
22
27
26
27
26
20
20
23
22
24
24
22
23
24
47
45
45
43
31
26
34
30
26
26
25
26
26
26
46
37
46
26
27
25
25
24
25
28
20
24
24
24
21
46
41
41
28
31
25
25
44
20
22
24
27
24
25
27
26
23
24
23
23
24
24
23
24
23
24
38
26
52
48
21
32
26
26
25
27
27
26
27
27
27
27
28
23
23
24
23
29
30
34
20
23
21
21
23
21
22
20
21
27
24
20
20
41
36
39
37
36
20
23
21
33
31
29
25
38
19
43
24
26
45
44
38
33
52
20
24
42
27
19
26
21
21
22
20
20
20
21
41
44
23
38
37
42
20
20
45
24
42
32
31
48
39
33
27
36
29
44
30
28
47
36
36
23
49
48
24
52
32
50
28
47
43
29
50
21
21
22
21
22
22
22
22
20
22
22
22
21
22
31
28
35
40
37
32
47
30
50
18
31
32
24
24
23
31
29
29
18
21
22
22
2

24
41
21
21
27
24
20
22
22
21
21
45
43
54
46
35
25
22
32
34
22
30
27
23
23
36
26
29
31
35
23
31
33
41
33
30
22
22
24
22
58
49
21
20
32
24
21
20
20
20
20
20
23
32
34
48
22
25
23
23
24
23
32
48
27
20
49
22
23
24
24
24
24
24
24
21
21
20
22
25
20
24
24
23
32
31
34
28
26
23
23
23
23
41
45
46
0
length
21
21
21
22
20
22
21
21
20
27
27
27
21
32
23
31
25
25
49
41
24
25
24
24
24
24
24
24
31
31
25
27
36
44
45
23
23
24
24
30
26
31
24
27
25
25
25
24
25
25
24
23
23
16
20
32
44
22
45
24
34
29
27
27
27
25
20
22
28
31
32
30
43
32
25
22
41
55
25
25
25
25
22
39
22
27
55
37
22
21
20
22
21
22
19
22
21
27
46
20
20
21
20
20
32
52
24
21
40
40
22
51
32
34
59
51
23
24
25
24
27
24
24
24
44
22
27
27
22
26
26
32
26
26
24
25
31
36
38
23
23
29
27
21
25
24
24
24
22
22
45
32
36
49
26
24
27
26
38
33
46
26
31
31
29
50
21
28
24
21
24
26
24
24
44
49
28
30
22
30
34
34
26
26
26
26
26
26
22
29
34
40
25
35
49
28
29
54
54
22
22
28
24
54
21
22
22
21
34
21
29
34
22
51
52
22
18
18
19
21
22
22
21
22
21
22
22
27
27
27
27
23
30
20
3

25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
44
27
22
26
20
24
27
25
27
25
25
25
31
23
24
26
43
37
36
30
20
26
26
38
19
40
54
54
22
23
23
23
27
26
27
24
28
44
24
36
35
27
21
25
23
30
29
45
35
27
44
40
25
32
22
21
25
33
54
30
23
23
22
53
36
20
20
24
23
31
46
22
24
43
23
29
21
24
36
36
32
23
23
47
32
29
34
24
26
24
21
23
28
26
28
20
25
20
24
32
20
40
23
47
20
42
25
24
22
47
25
25
24
25
20
46
30
30
21
23
31
36
23
45
27
24
24
51
27
42
29
33
32
45
35
37
23
24
56
17
18
23
24
23
29
30
42
32
39
40
37
32
46
48
22
24
27
22
58
30
46
34
55
35
46
24
40
24
24
25
25
24
25
25
24
25
24
25
24
24
24
24
25
24
24
24
24
24
24
24
24
25
24
24
51
22
24
24
27
24
47
24
42
20
24
29
25
27
35
27
59
35
43
25
29
25
49
49
24
27
39
27
33
26
27
22
42
21
21
22
21
22
22
22
26
25
33
23
27
31
23
30
34
30
32
20
19
22
22
21
25
35
20
23
22
23
24
22
49
20
22
22
21
22
25
27
43
20
19
21
19
22
22
25
42
22
25
27
27
29
34
20
32
22
26
36
33
23
18
25
22
22
30
22
20
23
46
27
25
46
24
25
20
21
21
22
22
2

In [5]:

print (data_added)

1871


In [ ]:
import pandas
print (pandas.DataFrame(allApartments))

In [ ]:
# Make a query to the specific DB and Collection
cursor = database.etuovi.find()

# Expand the cursor and construct the DataFrame
df =  pd.DataFrame(list(cursor))
del df['_id']
del df['condition']
del df['deposit']
del df['allow_pets']
print (df)